In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999


zip_ref = zipfile.ZipFile("F:\\project\\dlaicourse\\tmp\\happy-or-sad.zip", 'r')
zip_ref.extractall("F:\\project\\dlaicourse\\tmp\\h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACCURACY):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [12]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,'relu'),
    tf.keras.layers.Dense(1,'sigmoid')
])

from tensorflow.keras.optimizers import RMSprop,Adam

opt = Adam(learning_rate=0.0001)

model.compile(loss = 'binary_crossentropy',optimizer = opt,metrics=['accuracy'])

In [13]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1 / 255.0)

train_generator = train_datagen.flow_from_directory(
        "F:\\project\\dlaicourse\\tmp\\h-or-s",  
        target_size=(150, 150), 
        batch_size=32,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [14]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator, 
      epochs=20,
      verbose=1,
      callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
3/3 [==============================] - 3s 998ms/step - loss: 0.5804 - accuracy: 0.6125
Epoch 2/20
3/3 [==============================] - 1s 468ms/step - loss: 0.0979 - accuracy: 0.9500
Epoch 3/20
2/3 [===================>..........] - ETA: 0s - loss: 0.0440 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
3/3 [==============================] - 1s 459ms/step - loss: 0.0426 - accuracy: 1.0000
